In [ ]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations

AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
SC=["1","2","3","4","5","6","7"]


### Side chain
original  
ACDEFGHIKLMNPQRSTVWY  
12445131311222322155




In [ ]:
dipep = [''.join(i) for i in itertools.product(AA, repeat = 2)]
di_sc = [''.join(i) for i in itertools.product(SC, repeat = 3)]
print(di_sc)

In [ ]:
f_num=350
myseq="AILMVNQSTGPCHKPDEFWY"
trantab2=myseq.maketrans("AILMVNQSTGPCHKPDEFWY","11111222233455566777")
arr = numpy.empty((0,f_num), dtype=numpy.float)
class_arr = numpy.empty((0), dtype=numpy.int)
#print(arr.shape)
#print(arr)
all_fasta=("../major_capsid.fasta","../minor_capsid.fasta","../baseplate.fasta",
          "../major_tail.fasta","../minor_tail.fasta","../portal.fasta",
          "../tail_fiber.fasta","../tail_sheath.fasta","../collar.fasta",
          "../htj.fasta")
#print(all_fasta)
prot_class=0;
for file in all_fasta:
    print('####################' + file)
    for record in SeqIO.parse(file, "fasta"):
        #print(record.id)
        ll=len(record.seq)
        #xx=Seq(str(record.seq),IUPAC.protein)
        X = ProteinAnalysis(record.seq.__str__().replace('X','A').replace('J','L'))
        #X = ProteinAnalysis(str(xx))
        tt= [X.isoelectric_point(), X.instability_index(),ll,X.aromaticity(),
            X.molar_extinction_coefficient()[0],X.molar_extinction_coefficient()[1],
            X.gravy()]
        tt_n = numpy.asarray(tt,dtype=numpy.float)
        myseq=str(record.seq).translate(trantab2)
        dipep_count=[myseq.count(i)/ll for i in di_sc]
        dipep_count_n = numpy.asarray(dipep_count,dtype=numpy.float)
    
        cat_n= numpy.append(dipep_count_n,tt_n)
        cat_n = cat_n.reshape((1,cat_n.shape[0]))

        arr = numpy.append(arr,cat_n , axis=0)
        class_arr = numpy.append(class_arr,prot_class)
    prot_class+=1
    

    

In [ ]:
 print(arr.shape)

#arr_z = numpy.apply_over_axis(stats.zscore,arr,0)
arr_z=numpy.apply_along_axis(stats.zscore,0,arr)
#arr_z=numpy.apply_along_axis(stats.zscore,0,arr)
#print(arr_z)
#print(arr)
#print(arr_z.shape)
#print(prot_class)

In [ ]:
mean_arr=numpy.apply_along_axis(numpy.mean,0,arr)
#print(mean_arr)
print(mean_arr.shape)
std_arr=numpy.apply_along_axis(numpy.std,0,arr)
print(std_arr.shape)

In [ ]:
#targets = numpy.array([[2, 3, 4, 0]]).reshape(-1)

nb_classes = 10
one_hot_targets = numpy.eye(nb_classes)[class_arr]
print(one_hot_targets)
print(one_hot_targets.shape)

In [ ]:

final = numpy.concatenate((arr_z, one_hot_targets), axis=1)
final_raw = numpy.concatenate((arr, one_hot_targets), axis=1)

In [ ]:
#numpy.savetxt('test.out', final, delimiter='\t') 
#numpy.savetxt('test2.out', final_raw, delimiter='\t') 

In [ ]:
numpy.random.shuffle(final)
numpy.random.shuffle(final_raw)

In [ ]:
import pickle
##number of proteins for training
tt=25000
##number of features

pickle.dump(final[0:tt,0:f_num], open( "di_sc_train_x.p", "wb" ) )
pickle.dump(final[tt:,0:f_num],  open( "di_sc_test_x.p" , "wb" ) )
pickle.dump(final[0:tt,f_num:], open( "di_sc_train_y.p", "wb" ) )
pickle.dump(final[tt:,f_num:],  open( "di_sc_test_y.p" , "wb" ) )
pickle.dump(mean_arr,  open( "di_sc_mean.p" , "wb" ) )
pickle.dump(std_arr,  open( "di_sc_std.p" , "wb" ) )

In [ ]:
#pickle.dump(final_raw[0:tt,0:f_num], open( "dipep_raw_new_train_x.p", "wb" ) )
#pickle.dump(final_raw[tt:,0:f_num],  open( "dipep_raw_new_test_x.p" , "wb" ) )
#pickle.dump(final_raw[0:tt,f_num:], open( "dipep_raw_new_train_y.p", "wb" ) )
#pickle.dump(final_raw[tt:,f_num:],  open( "dipep_raw_new_test_y.p" , "wb" ) )